#GPT

In [ ]:
!pip install langchain==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 763.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import os
import pandas as pd
import requests
from langchain.prompts import PromptTemplate

In [ ]:
# Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Настройка путей
base_dir = "/content/drive/My Drive/TM-LLM"
input_folder = os.path.join(base_dir, 'texts')
data_folder = os.path.join(base_dir, 'data')

# Загрузка данных
df = pd.read_csv(os.path.join(data_folder, "topic_significant_entries_7_0.8.csv"))

In [ ]:
# API настройки
url = "https://forgetapi.ru/v2/gpt"
api_key = "forgetapi_key"  # Замените на ваш ключ API
headers = {
    "Content-Type": "application/json",
    "Authorization": api_key
}

In [ ]:

# Загрузка данных
df = pd.read_csv(os.path.join(data_folder, "topic_significant_entries.csv"))

# Функция для отправки запроса к API
def send_api_request(prompt):
    data = {
        "model": "gpt-4o",
        "messages": [
            {"content": "Вы - специалист по истории Российской Империи XIX века, умеющий обобщать и анализировать тексты.", "role": "system"},
            {"content": prompt, "role": "user"}
        ],
        "temperature": 0.7,
        "max_tokens": 2200
    }

    response = requests.post(url, headers=headers, json=data)
    print(f"Full API response: {response.text}")  # Для отладки

    if response.status_code == 200:
        response_json = response.json()
        if isinstance(response_json, dict) and "response" in response_json:
            return response_json["response"]
        elif isinstance(response_json, str):
            return response_json
        else:
            print(f"Unexpected response format: {response_json}")
            return str(response_json)
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None

# Шаблон промпта
prompt_template = PromptTemplate(
    input_variables=["documents", "keywords"],
    template="""
    В дневнике гимназиста середины XIX века при помощи тематического моделирования была
    выделена тема, которая содержит следующие наиболее релевантные дневниковые записи: {documents}. Дневниковые записи разделены символами "###"
    #Тема описывается следующими ключевыми словами: {keywords}.
    На основе приведенной выше информации о теме, пожалуйста, выполните следующие задачи:
    1. Сформулируй название темы (не более 10 слов), которое отражает ее суть.
    2. Составьте краткое описание темы (50-70 слов), раскрывающее ее основное содержание и контекст.
    Ваш ответ должен быть структурирован следующим образом:
    Название: [Ваше краткое название темы]
    Описание: [Ваше краткое описание темы]
    Пожалуйста, убедитесь, что вы возвращаете только название и описание в указанном формате, без дополнительных комментариев или пояснений.
    """
)

# Функция для обработки темы
def process_topic(topic_number):
    topic_df = df[df['Topic'] == topic_number]
    if topic_df.empty:
        return f"No data found for topic {topic_number}\n\n"

    documents = "###".join(topic_df['Text'].tolist())
    keywords = ", ".join(topic_df['Keywords'].iloc[0].split(", ")[:10])  # Берем первые 10 ключевых слов

    prompt = prompt_template.format(documents=documents, keywords=keywords)
    response = send_api_request(prompt)

    if response:
        result = f"Анализ темы {topic_number}:\n{response}\n\n{'='*50}\n\n"
        print(result)
        return result
    else:
        return f"Failed to get response for topic {topic_number}\n\n"

# Основной цикл обработки тем
def main():
    unique_topics = df['Topic'].unique()
    all_results = []

    for topic in unique_topics:
        try:
            result = process_topic(topic)
            all_results.append(result)
        except Exception as e:
            all_results.append(f"Error processing topic {topic}: {str(e)}\n\n")

    # Сохранение всех результатов в один файл
    with open(os.path.join(data_folder, "all_topics_analysis_gpt-4o_10_0.8_t0.7_1.txt"), "w", encoding="utf-8") as f:
        f.write("".join(all_results))

    print("Анализ всех тем завершен. Результаты сохранены в файл 'all_topics_analysis.txt'.")

if __name__ == "__main__":
    main()

Full API response: "Название: Семейная жизнь и досуг гимназиста в Петербурге и Царском Селе\n\nОписание: Тема охватывает повседневную жизнь гимназиста середины XIX века, его отношения с семьёй, поездки между Петербургом и Царским Селом, а также увлечения, такие как ловля рыбы и прогулки. Записи также включают описание погодных условий и различных событий, происходящих в жизни гимназиста и его близких."
Анализ темы 1:
Название: Семейная жизнь и досуг гимназиста в Петербурге и Царском Селе

Описание: Тема охватывает повседневную жизнь гимназиста середины XIX века, его отношения с семьёй, поездки между Петербургом и Царским Селом, а также увлечения, такие как ловля рыбы и прогулки. Записи также включают описание погодных условий и различных событий, происходящих в жизни гимназиста и его близких.



Full API response: "Название: Университетские волнения и правительственные меры в 1861 году\n\nОписание: Записи гимназиста середины XIX века описывают студенческие волнения в университетах Росс

# OpenAI gpt-4-turbo

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from openai import OpenAI
from langchain.prompts import PromptTemplate

# Настройка путей
base_dir = "/content/drive/My Drive/TM-LLM"
input_folder = os.path.join(base_dir, 'texts')
data_folder = os.path.join(base_dir, 'data')

# Загрузка данных
df = pd.read_csv(os.path.join(data_folder, "topic_significant_entries.csv"))

In [ ]:
# API настройки
PROXY_API_KEY = "PROXY_API_KEY"  # Замените на ваш ключ API ProxyAPI

client = OpenAI(
    api_key=PROXY_API_KEY,
    base_url="https://api.proxyapi.ru/openai/v1",
)

In [ ]:
# Функция для отправки запроса к API

def send_api_request(prompt):
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "Ты - специалист по истории Российской Империи XIX века, умеющий обобщать и анализировать тексты."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,  # Настройка температуры (значение от 0 до 2)
            max_tokens=5000    # Максимальное число токенов в ответе
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error in API request: {str(e)}")
        return None

# Шаблон промпта
prompt_template = PromptTemplate(
    input_variables=["documents", "keywords"],
    template="""
    В дневнике гимназиста середины XIX века при помощи тематического моделирования была
    выделена тема, которая содержит следующие наиболее релевантные дневниковые записи: {documents}. Дневниковые записи разделены символами "###"
    #Тема описывается следующими ключевыми словами: {keywords}.
    На основе приведенной выше информации о теме, пожалуйста, выполните следующие задачи:
    1. Сформулируй название темы (не более 10 слов), которое отражает ее суть.
    2. Составьте краткое описание темы (30-50 слов), раскрывающее ее основное содержание и контекст.
    Ваш ответ должен быть структурирован следующим образом:
    Название: [Ваш краткий лейбл темы]
    Описание: [Ваше краткое описание темы]
    Пожалуйста, убедитесь, что вы возвращаете только название и описание в указанном формате, без дополнительных комментариев или пояснений.
    """
)

# Функция для обработки темы
def process_topic(topic_number):
    topic_df = df[df['Topic'] == topic_number]
    if topic_df.empty:
        return f"No data found for topic {topic_number}\n\n"

    documents = "###".join(topic_df['Text'].tolist())
    keywords = ", ".join(topic_df['Keywords'].iloc[0].split(", ")[:10])  # Берем первые 10 ключевых слов

    prompt = prompt_template.format(documents=documents, keywords=keywords)
    response = send_api_request(prompt)

    if response:
        result = f"Анализ темы {topic_number}:\n{response}\n\n{'='*50}\n\n"
        print(result)
        return result
    else:
        return f"Failed to get response for topic {topic_number}\n\n"

# Основной цикл обработки тем
def main():
    unique_topics = df['Topic'].unique()
    all_results = []

    for topic in unique_topics:
        try:
            result = process_topic(topic)
            all_results.append(result)
        except Exception as e:
            all_results.append(f"Error processing topic {topic}: {str(e)}\n\n")

    # Сохранение всех результатов в один файл
    with open(os.path.join(data_folder, "all_topics_analysis_gpt-4-turbo2.txt"), "w", encoding="utf-8") as f:
        f.write("".join(all_results))

    print("Анализ всех тем завершен. Результаты сохранены в файл 'all_topics_analysis.txt'.")

if __name__ == "__main__":
    main()

#Anthropic

In [ ]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.3 MB/s eta 0:00:00


In [ ]:


import anthropic

client = anthropic.Anthropic(
    api_key = "proxyapi_key", # Замените на ваш ключ API
    base_url = "https://api.proxyapi.ru/anthropic",
)
message = client.messages.create(
    model="claude-3-opus-20240229", #claude-3-5-sonnet-20240620
    max_tokens=10,
    messages=[
        {"role": "user", "content": "Кто ты? Ответь одним предложением.0"}
    ]
)
print(message.content)

[TextBlock(text='Я - искусственный интелл', type='text')]


In [ ]:
import os
import pandas as pd
import anthropic
from langchain.prompts import PromptTemplate

# Настройка путей
base_dir = "/content/drive/My Drive/TM-LLM"
input_folder = os.path.join(base_dir, 'texts')
data_folder = os.path.join(base_dir, 'data')

# Загрузка данных
df = pd.read_csv(os.path.join(data_folder, "topic_significant_entries.csv"))


In [ ]:
df = pd.read_csv(os.path.join(data_folder, "topic_significant_entries_10_0.8.csv"))
import anthropic
# API настройки
PROXY_API_KEY = "sk-Nf6OoPa3TlTyBnxrd4mNlWC8QGlSS6wr"

client = anthropic.Anthropic(
    api_key=PROXY_API_KEY,
    base_url="https://api.proxyapi.ru/anthropic",
)

# Функция для отправки запроса к API

def send_api_request(prompt):
    try:
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            system="Вы - специалист по истории Российской Империи XIX века, умеющий обобщать и анализировать тексты.",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return message.content
    except Exception as e:
        print(f"Error in API request: {str(e)}")
        return None


# Шаблон промпта
prompt_template = PromptTemplate(
    input_variables=["documents", "keywords"],
    template="""
    В дневнике гимназиста середины XIX века при помощи тематического моделирования была
    выделена тема, которая содержит следующие наиболее релевантные дневниковые записи: {documents}. Дневниковые записи разделены символами "###"
    #Тема описывается следующими ключевыми словами: {keywords}.
    На основе приведенной выше информации о теме, пожалуйста, выполните следующие задачи:
    1. Сформулируй название темы (не более 10 слов), который отражает ее суть.
    2. Составьте краткое описание темы (50-70 слов), раскрывающее ее основное содержание и контекст.
    Ваш ответ должен быть структурирован следующим образом:
    Название: [Ваше краткое название темы]
    Описание: [Ваше краткое описание темы]
    Пожалуйста, убедитесь, что вы возвращаете только название и описание в указанном формате, без дополнительных комментариев или пояснений.
    """
)

# Функция для обработки темы
def process_topic(topic_number):
    topic_df = df[df['Topic'] == topic_number]
    if topic_df.empty:
        return f"No data found for topic {topic_number}\n\n"

    documents = "###".join(topic_df['Text'].tolist())
    keywords = ", ".join(topic_df['Keywords'].iloc[0].split(", ")[:10])  # Берем первые 10 ключевых слов

    prompt = prompt_template.format(documents=documents, keywords=keywords)
    response = send_api_request(prompt)

    if response:
        result = f"Анализ темы {topic_number}:\n{response}\n\n{'='*50}\n\n"
        print(result)
        return result
    else:
        return f"Failed to get response for topic {topic_number}\n\n"

# Основной цикл обработки тем
def main():
    unique_topics = df['Topic'].unique()
    all_results = []

    for topic in unique_topics:
        try:
            result = process_topic(topic)
            all_results.append(result)
        except Exception as e:
            all_results.append(f"Error processing topic {topic}: {str(e)}\n\n")

    # Сохранение всех результатов в один файл
    with open(os.path.join(data_folder, "all_topics_analysis_anthropic_3-5__10_0.8No_KW.txt"), "w", encoding="utf-8") as f:
        f.write("".join(all_results))

    print("Анализ всех тем завершен. Результаты сохранены в файл 'all_topics_analysis.txt'.")

if __name__ == "__main__":
    main()

Анализ темы 1:
[TextBlock(text='Название: Повседневная жизнь и досуг гимназиста в середине XIX века\n\nОписание: Тема отражает бытовые аспекты жизни гимназиста середины XIX века: семейные отношения, досуг, учебу и путешествия. Автор описывает поездки между Петербургом, Царским Селом и имением Грузино, занятия рыбалкой, встречи с родственниками. Упоминаются денежные отношения, погода, праздники. Дневник дает представление о распорядке дня, интересах и заботах подростка из образованной семьи того времени, а также о социальном окружении и материальном быте.', type='text')]



Анализ темы 2:
[TextBlock(text='Название: Студенческие волнения и закрытие Санкт-Петербургского университета в 1861 году\n\nОписание: Тема охватывает события вокруг студенческих протестов в Санкт-Петербургском университете осенью 1861 года. Описываются конфликты между студентами и администрацией, введение новых правил, сходки студентов, реакция властей, аресты активистов и закрытие университета. Отражены действия поп

#LLAMA

In [ ]:
import os
import pandas as pd
import requests
import json
from langchain.prompts import PromptTemplate

# Настройка путей
base_dir = "/content/drive/My Drive/TM-LLM"
input_folder = os.path.join(base_dir, 'texts')
data_folder = os.path.join(base_dir, 'data')

# Загрузка данных
df = pd.read_csv(os.path.join(data_folder, "topic_significant_entries_10_0.8.csv"))

# API настройки
API_KEY = "YOUR_API_KEY"  # Замените на ваш ключ API ForgetAPI
url = "https://forgetapi.ru/llama"

headers = {
    "Content-Type": "application/json",
    "Authorization": API_KEY
}

# Функция для отправки запроса к API
def send_api_request(prompt):
    try:
        data = {
            "messages": [
                {"content": "Вы - специалист по истории Российской Империи XIX века, умеющий обобщать и анализировать тексты.", "role": "system"},
                {"content": prompt, "role": "user"}
            ],
            "model": "llama-3.1-405b",
            "max_tokens": 2000,
            "temperature": 0.5
        }

        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            # Попробуем напечатать полный ответ для отладки
            print("Full API response:")
            print(json.dumps(response.json(), indent=2, ensure_ascii=False))

            # Попробуем получить содержимое ответа
            response_content = response.json()
            if isinstance(response_content, str):
                return response_content
            elif isinstance(response_content, dict):
                return response_content.get('response', str(response_content))
            else:
                print(f"Unexpected response type: {type(response_content)}")
                return str(response_content)
        else:
            print(f"Request failed with status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error in API request: {str(e)}")
        return None

# Шаблон промпта
prompt_template = PromptTemplate(
    input_variables=["documents", "keywords"],
    template="""
    В дневнике гимназиста середины XIX века при помощи тематического моделирования была
    выделена тема, которая содержит следующие наиболее релевантные дневниковые записи: {documents}. Дневниковые записи разделены символами "###"
    #Тема описывается следующими ключевыми словами: {keywords}.
    На основе приведенной выше информации о теме, пожалуйста, выполните следующие задачи:
    1. Сформулируй название темы (не более 10 слов), которое отражает ее суть.
    2. Составьте краткое описание темы (50-70 слов), раскрывающее ее основное содержание и контекст.
    Ваш ответ должен быть структурирован следующим образом:
    Название: [Ваше краткое название темы]
    Описание: [Ваше краткое описание темы]
    Пожалуйста, убедитесь, что вы возвращаете только название и описание в указанном формате, без дополнительных комментариев или пояснений.
    """
)

# Функция для обработки темы
def process_topic(topic_number):
    topic_df = df[df['Topic'] == topic_number]
    if topic_df.empty:
        return f"No data found for topic {topic_number}\n\n"

    documents = "###".join(topic_df['Text'].tolist())
    keywords = ", ".join(topic_df['Keywords'].iloc[0].split(", ")[:10])  # Берем первые 10 ключевых слов

    prompt = prompt_template.format(documents=documents, keywords=keywords)
    response = send_api_request(prompt)

    if response:
        result = f"Анализ темы {topic_number}:\n{response}\n\n{'='*50}\n\n"
        print(result)
        return result
    else:
        return f"Failed to get response for topic {topic_number}\n\n"

# Основной цикл обработки тем
def main():
    unique_topics = df['Topic'].unique()
    all_results = []

    for topic in unique_topics:
        try:
            result = process_topic(topic)
            all_results.append(result)
        except Exception as e:
            all_results.append(f"Error processing topic {topic}: {str(e)}\n\n")

    # Сохранение всех результатов в один файл
    with open(os.path.join(data_folder, "all_topics_analysis_llama_10_0.8_0.5_NOKW.txt"), "w", encoding="utf-8") as f:
        f.write("".join(all_results))

    print("Анализ всех тем завершен. Результаты сохранены в файл 'all_topics_analysis.txt'.")

if __name__ == "__main__":
    main()

Full API response:
"Название: Воспоминания и будни гимназиста XIX века\n\nОписание: Тема охватывает повседневную жизнь гимназиста в Российской Империи XIX века, включая семейные события, учебу, увлечения, такие как рыбалка, и впечатления от поездок, в частности, в Грузино. Записи также отражают заботы о погоде, здоровье и финансовых вопросах, давая представление о жизни и ценностях того времени."
Анализ темы 1:
Название: Воспоминания и будни гимназиста XIX века

Описание: Тема охватывает повседневную жизнь гимназиста в Российской Империи XIX века, включая семейные события, учебу, увлечения, такие как рыбалка, и впечатления от поездок, в частности, в Грузино. Записи также отражают заботы о погоде, здоровье и финансовых вопросах, давая представление о жизни и ценностях того времени.



Request failed with status code: 502
Full API response:
"Название: Внутренние переживания и стремления гимназиста\n\nОписание: Тема охватывает личные размышления и эмоции гимназиста, касающиеся его учебной